In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnull
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
spark

In [3]:
sc

<SparkContext master=spark://devenv:7077 appName=PySparkShell>

In [4]:
# Prepare data
logs = spark.read.parquet("hdfs://devenv/user/spark/spark_mllib_101/ec_web_logs_analysis/data/")

In [5]:
# Preprocessing and feature engineering
feature_prep = logs.select("product_category_id", "device_type", "connect_type", "age_group") \
                   .where(~isnull("age_group"))

feature_prep = StringIndexer(inputCol="age_group", outputCol="age_group_indexed").fit(feature_prep).transform(feature_prep)

final_data = VectorAssembler(inputCols=["product_category_id", "device_type", "connect_type"],
                             outputCol="features").transform(feature_prep)

In [6]:
# Split data into train and test sets
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [7]:
# Model training
classifier = RandomForestClassifier(featuresCol="features", labelCol="age_group_indexed", numTrees=20, maxDepth=10)
model = classifier.fit(train_data)

In [8]:
# Transform the test data using the model to get predictions
predicted_test_data = model.transform(test_data)

In [9]:
# Evaluate the model performance
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='age_group_indexed',
                                                       predictionCol='prediction',
                                                       metricName='accuracy')
print("Accuracy: {}", evaluator_accuracy.evaluate(predicted_test_data))

Accuracy: {} 0.8251280981499107


In [10]:
confusion_matrix_info = predicted_test_data.select("age_group_indexed", "prediction")\
                                           .groupBy("age_group_indexed", "prediction")\
                                           .count()

confusion_matrix_info.orderBy("age_group_indexed", "prediction").show()
confusion_matrix_info.orderBy("prediction", "age_group_indexed").show()

+-----------------+----------+-----+
|age_group_indexed|prediction|count|
+-----------------+----------+-----+
|              0.0|       0.0|40051|
|              0.0|       1.0| 3131|
|              0.0|       2.0| 2425|
|              0.0|       3.0| 2693|
|              1.0|       0.0| 1135|
|              1.0|       1.0|33908|
|              1.0|       2.0| 4289|
|              2.0|       0.0| 1229|
|              2.0|       1.0| 5483|
|              2.0|       2.0|32452|
|              3.0|       0.0| 3314|
|              3.0|       1.0| 1682|
|              3.0|       2.0| 1444|
|              3.0|       3.0|20162|
+-----------------+----------+-----+

+-----------------+----------+-----+
|age_group_indexed|prediction|count|
+-----------------+----------+-----+
|              0.0|       0.0|40051|
|              1.0|       0.0| 1135|
|              2.0|       0.0| 1229|
|              3.0|       0.0| 3314|
|              0.0|       1.0| 3131|
|              1.0|       1.0|33908|


In [11]:
# Save the model
model.save("hdfs://devenv/user/spark/spark_mllib_101/ec_web_logs_analysis/model_age_group_prediction/")


# +---------+-----------------+                                                   
# |age_group|age_group_indexed|
# +---------+-----------------+
# | under 20|              2.0|
# |  over 50|              3.0|
# |    21-35|              0.0|
# |    36-50|              1.0|
# +---------+-----------------+